Following Analysis are done on the data

1.) pandas_profiling import ProfileReport - Get the Basic Idea of the data and different relation
Inference: Features and Body Fat are related Linearly. 

2.)VIF - Checking for Multi Collinearity
Inference: Weight , Abdomen, Hip are having high collinearity with Density.

3.)PCA check the Variance Spread
Inference: 5 Principle component account for 85% of Variance.

4.)Hypothesis Testing Feature Identification by Checking Slope in Population
Inference: Density satisfies the 5% signficance value. Age, Chest, Weight though they are not satisfying the Required confidence interval of 95% they are having relatively better t value

In [1]:
import pandas as pd
import numpy as np

In [2]:
Dataset_Train = pd.read_csv(r'C:\Users\gvi7cob\Documents\AI\BodyFat_Prediction\bodyfat.csv')

In [3]:
Dataset_Train.head()

,Density,BodyFat,Age,Weight,Height,Neck,Chest,Abdomen,Hip,Thigh,Knee,Ankle,Biceps,Forearm,Wrist
0,1.0708,12.3,23,154.25,67.75,36.2,93.1,85.2,94.5,59.0,37.3,21.9,32.0,27.4,17.1
1,1.0853,6.1,22,173.25,72.25,38.5,93.6,83.0,98.7,58.7,37.3,23.4,30.5,28.9,18.2
2,1.0414,25.3,22,154.00,66.25,34.0,95.8,87.9,99.2,59.6,38.9,24.0,28.8,25.2,16.6
3,1.0751,10.4,26,184.75,72.25,37.4,101.8,86.4,101.2,60.1,37.3,22.8,32.4,29.4,18.2
4,1.0340,28.7,24,184.25,71.25,34.4,97.3,100.0,101.9,63.2,42.2,24.0,32.2,27.7,17.7


In [4]:
from pandas_profiling import ProfileReport

In [5]:
profile = ProfileReport(Dataset_Train)

In [6]:
profile.to_widgets()

Summarize dataset:   0%|          | 0/29 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
profile.to_file("EDA_BodyFat_Profile.html")

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Perform VIF (Variance Inflation Factor)
To check if there is any mulitcollinearity between different Features (Since the data is taken from the human body. there is a good chance that sizes are relatable to each other)

# VIF - Checking for Multi Collinearity (Self Code without stats model)

In [8]:
Indep_Varia = Dataset_Train.drop(['BodyFat'],axis=1,inplace=False)

In [9]:
Indep_Varia.head()

,Density,Age,Weight,Height,Neck,Chest,Abdomen,Hip,Thigh,Knee,Ankle,Biceps,Forearm,Wrist
0,1.0708,23,154.25,67.75,36.2,93.1,85.2,94.5,59.0,37.3,21.9,32.0,27.4,17.1
1,1.0853,22,173.25,72.25,38.5,93.6,83.0,98.7,58.7,37.3,23.4,30.5,28.9,18.2
2,1.0414,22,154.00,66.25,34.0,95.8,87.9,99.2,59.6,38.9,24.0,28.8,25.2,16.6
3,1.0751,26,184.75,72.25,37.4,101.8,86.4,101.2,60.1,37.3,22.8,32.4,29.4,18.2
4,1.0340,24,184.25,71.25,34.4,97.3,100.0,101.9,63.2,42.2,24.0,32.2,27.7,17.7


In [10]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt

In [11]:
Coffient_VIFCalc = pd.DataFrame(index=Indep_Varia.columns)
for feature in Indep_Varia:
    Coffient_VIFCalc[feature] = 0.0

def Calc_VIF(OtherInV, InV_Ch,var):
    regr = Ridge(alpha = 0)
    regr.fit(OtherInV, InV_Ch)
    score = regr.score(OtherInV, InV_Ch)
    if(score > 0.9):
        for feature, i in zip(OtherInV, regr.coef_):
            Coffient_VIFCalc[var][feature] = i

    #print(f'R^2 for {var}={regr.score(OtherInV, InV_Ch)}')
    #print(f'VIF for {var}={(1/(1-score))}')
    #print(f'Coffient for {var}={regr.coef_}')
    return (1/(1-score))

In [12]:
VIF_value = pd.DataFrame()
VIF_value['Feature'] = Indep_Varia.columns
VIF_value['Score'] = [Calc_VIF(Indep_Varia.drop([var],axis=1,inplace=False),Indep_Varia[var],var) for var in Indep_Varia.columns]

In [13]:
VIF_value

,Feature,Score
0,Density,3.818257
1,Age,2.274699
2,Weight,34.031683
3,Height,1.677765
4,Neck,4.396475
5,Chest,9.472241
6,Abdomen,18.119895
7,Hip,14.960951
8,Thigh,7.887715
9,Knee,4.612291


In [14]:
print(VIF_value[VIF_value['Score']>10].Feature)

2     Weight
6    Abdomen
7        Hip
Name: Feature, dtype: object


In [15]:
print(Coffient_VIFCalc)

         Density  Age     Weight  Height  Neck  Chest     Abdomen        Hip  \
Density      0.0  0.0  64.084220     0.0   0.0    0.0 -154.003799  19.936700   
Age          0.0  0.0  -0.115680     0.0   0.0    0.0    0.090972  -0.015146   
Weight       0.0  0.0   0.000000     0.0   0.0    0.0    0.118231   0.191426   
Height       0.0  0.0   0.791785     0.0   0.0    0.0   -0.056061  -0.167280   
Neck         0.0  0.0   1.180063     0.0   0.0    0.0    0.211338  -0.296155   
Chest        0.0  0.0   0.736917     0.0   0.0    0.0    0.368205  -0.124348   
Abdomen      0.0  0.0   0.467621     0.0   0.0    0.0    0.000000   0.190167   
Hip          0.0  0.0   1.416225     0.0   0.0    0.0    0.355717   0.000000   
Thigh        0.0  0.0   0.189247     0.0   0.0    0.0    0.053780   0.328920   
Knee         0.0  0.0   1.138000     0.0   0.0    0.0   -0.132275   0.071430   
Ankle        0.0  0.0   0.787631     0.0   0.0    0.0   -0.251190  -0.066806   
Biceps       0.0  0.0   0.592707     0.0

Weight , Abdomen, Hip are having high relation with Density.

In [16]:
correlation = Dataset_Train['Density'].corr(Dataset_Train['BodyFat'])
print(f'Correlation b.w Depedant Variable Density and BodyFat is {correlation}')

Correlation b.w Depedant Variable Density and BodyFat is -0.9877824021639879


In [17]:
for feature in VIF_value[VIF_value['Score']>10].Feature:
    correlation = Dataset_Train['BodyFat'].corr(Dataset_Train[feature])
    print(f'Correlation b.w Depedant Variable BodyFat and {feature} is {correlation}')

Correlation b.w Depedant Variable BodyFat and Weight is 0.6124140022026479
Correlation b.w Depedant Variable BodyFat and Abdomen is 0.8134322847810495
Correlation b.w Depedant Variable BodyFat and Hip is 0.6252009175086634


We can see from VIF High Collinearity exist between Density and Weight ,Abdomen ,Hip. We have higher correlation between Density and Dependant variable BodyFat. So we can consider removing these feature (Weight ,Abdomen ,Hip) from our dataset

We can Drop these three feature and check for VIF

In [18]:
VIF_Dropped_IndepVariaable = Indep_Varia.copy()

In [19]:
VIF_Dropped_IndepVariaable.drop(['Weight','Abdomen','Hip'], axis = 1,inplace = True)

In [20]:
VIF_Dropped_IndepVariaable

,Density,Age,Height,Neck,Chest,Thigh,Knee,Ankle,Biceps,Forearm,Wrist
0,1.0708,23,67.75,36.2,93.1,59.0,37.3,21.9,32.0,27.4,17.1
1,1.0853,22,72.25,38.5,93.6,58.7,37.3,23.4,30.5,28.9,18.2
2,1.0414,22,66.25,34.0,95.8,59.6,38.9,24.0,28.8,25.2,16.6
3,1.0751,26,72.25,37.4,101.8,60.1,37.3,22.8,32.4,29.4,18.2
4,1.0340,24,71.25,34.4,97.3,63.2,42.2,24.0,32.2,27.7,17.7
...,...,...,...,...,...,...,...,...,...,...,...
247,1.0736,70,67.00,34.9,89.2,49.6,34.8,21.5,25.6,25.7,18.5
248,1.0236,72,69.75,40.9,108.5,59.6,40.8,23.2,35.2,28.6,20.1
249,1.0328,72,66.00,38.9,111.1,60.3,37.3,21.5,31.3,27.2,18.0
250,1.0399,72,70.50,38.9,108.3,56.0,41.6,22.7,30.5,29.4,19.8


Checking if the multi collinearity exist and after the removal of 'Weight','Abdomen','Hip' Feature

In [21]:
Coffient_VIFCalc_dropped = pd.DataFrame(index=VIF_Dropped_IndepVariaable.columns)
for feature in VIF_Dropped_IndepVariaable:
    Coffient_VIFCalc[feature] = 0.0

def Calc_VIF(OtherInV, InV_Ch,var):
    regr = Ridge(alpha = 0)
    regr.fit(OtherInV, InV_Ch)
    score = regr.score(OtherInV, InV_Ch)
    if(score > 0.9):
        for feature, i in zip(OtherInV, regr.coef_):
            Coffient_VIFCalc_dropped[var][feature] = i

    #print(f'R^2 for {var}={regr.score(OtherInV, InV_Ch)}')
    #print(f'VIF for {var}={(1/(1-score))}')
    #print(f'Coffient for {var}={regr.coef_}')
    return (1/(1-score))

In [22]:
VIF_value = pd.DataFrame()
VIF_value['Feature'] = VIF_Dropped_IndepVariaable.columns
VIF_value['Score'] = [Calc_VIF(VIF_Dropped_IndepVariaable.drop([var],axis=1,inplace=False),VIF_Dropped_IndepVariaable[var],var) for var in VIF_Dropped_IndepVariaable.columns]

In [23]:
VIF_value

,Feature,Score
0,Density,2.455780
1,Age,2.017710
2,Height,1.343092
3,Neck,3.948853
4,Chest,4.692214
5,Thigh,5.718672
6,Knee,4.067145
7,Ankle,1.835964
8,Biceps,3.479859
9,Forearm,2.152666


In [24]:
print(VIF_value[VIF_value['Score']>10].Feature)

Series([], Name: Feature, dtype: object)


As we can see that there are no feature with Higher VIF

In [25]:
print(Coffient_VIFCalc_dropped)

Empty DataFrame
Columns: []
Index: [Density, Age, Height, Neck, Chest, Thigh, Knee, Ankle, Biceps, Forearm, Wrist]


# PCA check the Information Spread

In [26]:
from sklearn.decomposition import PCA
pca = PCA()

In [27]:
Indep_Varia.head()

,Density,Age,Weight,Height,Neck,Chest,Abdomen,Hip,Thigh,Knee,Ankle,Biceps,Forearm,Wrist
0,1.0708,23,154.25,67.75,36.2,93.1,85.2,94.5,59.0,37.3,21.9,32.0,27.4,17.1
1,1.0853,22,173.25,72.25,38.5,93.6,83.0,98.7,58.7,37.3,23.4,30.5,28.9,18.2
2,1.0414,22,154.00,66.25,34.0,95.8,87.9,99.2,59.6,38.9,24.0,28.8,25.2,16.6
3,1.0751,26,184.75,72.25,37.4,101.8,86.4,101.2,60.1,37.3,22.8,32.4,29.4,18.2
4,1.0340,24,184.25,71.25,34.4,97.3,100.0,101.9,63.2,42.2,24.0,32.2,27.7,17.7


In [28]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled = scaler.fit_transform(Indep_Varia)
columns = Indep_Varia.columns

In [29]:
df_pca = pd.DataFrame(scaled, columns=Indep_Varia.columns, index=Indep_Varia.index)

In [30]:
df_pca

,Density,Age,Weight,Height,Neck,Chest,Abdomen,Hip,Thigh,Knee,Ankle,Biceps,Forearm,Wrist
0,0.665496,0.016949,0.146127,0.792746,0.253731,0.242531,0.200762,0.151515,0.294264,0.267081,0.189189,0.356436,0.460432,0.232143
1,0.792801,0.000000,0.223789,0.886010,0.368159,0.251318,0.172808,0.218501,0.286783,0.267081,0.290541,0.282178,0.568345,0.428571
2,0.407375,0.000000,0.145105,0.761658,0.144279,0.289982,0.235070,0.226475,0.309227,0.366460,0.331081,0.198020,0.302158,0.142857
3,0.703248,0.067797,0.270795,0.886010,0.313433,0.395431,0.216010,0.258373,0.321696,0.267081,0.250000,0.376238,0.604317,0.428571
4,0.342406,0.033898,0.268751,0.865285,0.164179,0.316344,0.388818,0.269537,0.399002,0.571429,0.331081,0.366337,0.482014,0.339286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,0.690079,0.813559,0.064378,0.777202,0.189055,0.173989,0.180432,0.060606,0.059850,0.111801,0.162162,0.039604,0.338129,0.482143
248,0.251097,0.847458,0.337216,0.834197,0.487562,0.513181,0.452351,0.311005,0.309227,0.484472,0.277027,0.514851,0.546763,0.767857
249,0.331870,0.847458,0.278970,0.756477,0.388060,0.558875,0.534943,0.266348,0.326683,0.267081,0.162162,0.321782,0.446043,0.392857
250,0.394205,0.847458,0.295320,0.849741,0.388060,0.509666,0.405337,0.204147,0.219451,0.534161,0.243243,0.282178,0.604317,0.714286


In [31]:
pca.fit(df_pca)

PCA()

In [32]:
print(f'VarianceRatio:{pca.explained_variance_ratio_}')
print(f'SingularValues:{pca.singular_values_}')

VarianceRatio:[0.56303811 0.18208083 0.0804002  0.04308037 0.02725809 0.02345918
 0.02019563 0.0168645  0.01616196 0.01078613 0.00845671 0.00447147
 0.00257746 0.00116935]
SingularValues:[6.35902887 3.61621312 2.40298173 1.75898287 1.39916707 1.29801089
 1.20434422 1.10054713 1.07738006 0.880146   0.77933206 0.5666921
 0.43024712 0.28979777]


In [33]:
#Number of Feature is less than Samples so we take number of PCA components as Length of Feature
index = ['pca_%i' % i for i in range(len(Indep_Varia.columns))]
pc_eigen_vector = pd.DataFrame(pca.components_, columns=Indep_Varia.columns, index=index)

In [34]:
print(pc_eigen_vector)

         Density       Age    Weight    Height      Neck     Chest   Abdomen  \
pca_0  -0.286183  0.057276  0.288353  0.042878  0.258159  0.338023  0.312728   
pca_1  -0.226181  0.914310 -0.063036 -0.085079  0.004405  0.070513  0.108252   
pca_2   0.634693  0.198646 -0.009976  0.194700  0.140691 -0.113814 -0.229498   
pca_3   0.142145  0.001498  0.116964  0.092515 -0.085253 -0.020937  0.080979   
pca_4  -0.389339  0.009708 -0.124108  0.004717 -0.301964 -0.220079 -0.125702   
pca_5  -0.259767 -0.271895  0.018559  0.089040  0.212400  0.227443  0.149930   
pca_6  -0.042662  0.032448 -0.080049 -0.270404 -0.012420 -0.070977 -0.105063   
pca_7   0.396050  0.090450  0.120188 -0.179124  0.168358  0.484100  0.146987   
pca_8   0.105210 -0.035067 -0.104809 -0.833883 -0.051056 -0.093682  0.000654   
pca_9   0.110769 -0.065289  0.037163 -0.036465 -0.790042  0.451771  0.093927   
pca_10  0.104470  0.165830  0.133631  0.310172 -0.319072 -0.241186  0.137263   
pca_11  0.041148 -0.047696  0.193572 -0.

# Hypothesis Testing for Checking Slope in Population

In [35]:
SlopeCheck_dataframe = Dataset_Train.copy()

In [36]:
Dataset_Train.shape

(252, 15)

In [37]:
SlopeCheck_dataframe.shape

(252, 15)

We have 252 sample and 14 Features. Degree of Freedom is 238. Considering a significance value of 5% we will have T-test value 1.984 (at 95% confidence level)

In [38]:
import statsmodels.api as sm
import numpy as np

In [39]:
#We can check the relation between Abdomen with other Body parts.

SlopeCheck_dataframe['ArmbyAbdomen'] = (Dataset_Train['Biceps']+Dataset_Train['Forearm']+Dataset_Train['Wrist'])/Dataset_Train['Abdomen']
SlopeCheck_dataframe['LegbyAbdomen'] = (Dataset_Train['Thigh']+Dataset_Train['Knee']+Dataset_Train['Ankle'])/Dataset_Train['Abdomen']
SlopeCheck_dataframe['NeckbyAbdomen'] = Dataset_Train['Neck']/Dataset_Train['Abdomen']

In [40]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled = scaler.fit_transform(SlopeCheck_dataframe)
columns = Indep_Varia.columns
SlopeCheck_dataframe = pd.DataFrame(scaled, columns=SlopeCheck_dataframe.columns, index=SlopeCheck_dataframe.index)


In [41]:
Y = SlopeCheck_dataframe['BodyFat']
X = SlopeCheck_dataframe.drop('BodyFat',axis=1,inplace=False)
X = sm.add_constant(X)

In [42]:
X

,const,Density,Age,Weight,Height,Neck,Chest,Abdomen,Hip,Thigh,Knee,Ankle,Biceps,Forearm,Wrist,ArmbyAbdomen,LegbyAbdomen,NeckbyAbdomen
0,1.0,0.801647,-1.740073,-0.841246,-0.656205,-0.738665,-0.918048,-0.683533,-0.755929,-0.077479,-0.536131,-0.710826,-0.090676,-0.626718,-1.212541,0.511710,0.727639,0.351725
1,1.0,1.565061,-1.819583,-0.193462,0.574790,0.209365,-0.858621,-0.887963,-0.168502,-0.134736,-0.536131,0.175947,-0.588143,0.117079,-0.031943,1.038541,1.260603,1.565183
2,1.0,-0.746240,-1.819583,-0.849769,-1.066536,-1.645475,-0.597144,-0.432643,-0.098570,0.037035,0.128592,0.530656,-1.151940,-1.717621,-1.749177,-0.834812,0.793234,-0.833918
3,1.0,1.028039,-1.501543,0.198617,0.574790,-0.244041,0.115976,-0.572026,0.181157,0.132464,-0.536131,-0.178762,0.041982,0.365012,-0.031943,0.910378,0.768001,0.600432
4,1.0,-1.135844,-1.660563,0.181570,0.301235,-1.480601,-0.418864,0.691719,0.279061,0.724121,1.499585,0.530656,-0.024347,-0.477959,-0.568578,-1.221349,-0.243238,-2.166640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,1.0,0.949065,1.996904,-1.523123,-0.861370,-1.274507,-1.381576,-0.832209,-1.553151,-1.871535,-1.574763,-0.947298,-2.213203,-1.469689,0.290039,-0.383476,-0.526765,0.120742
248,1.0,-1.683396,2.155924,0.752642,-0.109096,1.198613,0.912294,1.156332,0.642706,0.037035,0.917952,0.057711,0.970588,-0.031680,2.007273,-0.893646,-1.458938,-0.749209
249,1.0,-1.199023,2.155924,0.266805,-1.134925,0.374239,1.221312,1.760328,0.251088,0.170635,-0.536131,-0.947298,-0.322827,-0.725891,-0.246597,-2.499615,-2.592705,-2.014729
250,1.0,-0.825214,2.155924,0.403180,0.096070,0.374239,0.888523,0.812518,-0.294379,-0.650050,1.250314,-0.237880,-0.588143,0.365012,1.685292,-1.068188,-1.350548,-0.920952


In [43]:
model = sm.OLS(Y,X,hasconst=True)
results = model.fit()

In [44]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                BodyFat   R-squared:                       0.978
Model:                            OLS   Adj. R-squared:                  0.977
Method:                 Least Squares   F-statistic:                     615.6
Date:                Wed, 07 Jul 2021   Prob (F-statistic):          1.48e-183
Time:                        20:11:20   Log-Likelihood:                 124.08
No. Observations:                 252   AIC:                            -212.2
Df Residuals:                     234   BIC:                            -148.6
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const         -1.457e-16      0.010  -1.51e-14      1.000      -0.019       0.019
Density          -0.9361      0.019    -48.338      0.000      -0.974      -0.898
Age               0.0191      0.015      1.305      0.193      -0.010       0.048
Weight            0.0378      0.057      0.660      0.510      -0.075       0.151
Height           -0.0032      0.013     -0.249      0.804      -0.029       0.022
Neck             -0.0771      0.157     -0.491      0.624      -0.386       0.232
Chest             0.0269      0.030      0.889      0.375      -0.033       0.087
Abdomen           0.0423      0.126      0.337      0.736      -0.205       0.290
Hip               0.0165      0.038      0.431      0.667      -0.059       0.092
Thigh            -0.0137      0.105     -0.131      0.896      -0.220       0.192
Knee             -0.0033      0.049     -0.066      0.947      -0.100       0.093
Ankle            -0.0183      0.036     -0.510      0.610      -0.089       0.052
Biceps            0.0188      0.088      0.212      0.832      -0.156       0.193
Forearm           0.0308      0.053      0.582      0.561      -0.073       0.135
Wrist             0.0126      0.031      0.404      0.687      -0.049       0.074
ArmbyAbdomen     -0.0790      0.177     -0.447      0.655      -0.427       0.269
LegbyAbdomen      0.0045      0.172      0.026      0.979      -0.334       0.343
NeckbyAbdomen     0.0843      0.190      0.444      0.658      -0.290       0.459
==============================================================================
Omnibus:                      364.693   Durbin-Watson:                   2.117
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           100331.562
Skew:                           6.391   Prob(JB):                         0.00
Kurtosis:                      99.912   Cond. No.                         92.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [45]:
results.tvalues

const           -1.507303e-14
Density         -4.833772e+01
Age              1.304583e+00
Weight           6.598684e-01
Height          -2.485617e-01
Neck            -4.913061e-01
Chest            8.892057e-01
Abdomen          3.373649e-01
Hip              4.306306e-01
Thigh           -1.306461e-01
Knee            -6.627735e-02
Ankle           -5.104271e-01
Biceps           2.124744e-01
Forearm          5.818130e-01
Wrist            4.036491e-01
ArmbyAbdomen    -4.470143e-01
LegbyAbdomen     2.599502e-02
NeckbyAbdomen    4.436262e-01
dtype: float64

In [46]:
results.params

const           -1.457168e-16
Density         -9.361314e-01
Age              1.907602e-02
Weight           3.782100e-02
Height          -3.244590e-03
Neck            -7.707374e-02
Chest            2.692917e-02
Abdomen          4.234598e-02
Hip              1.648447e-02
Thigh           -1.365278e-02
Knee            -3.253506e-03
Ankle           -1.828771e-02
Biceps           1.879764e-02
Forearm          3.078223e-02
Wrist            1.259360e-02
ArmbyAbdomen    -7.896906e-02
LegbyAbdomen     4.468005e-03
NeckbyAbdomen    8.432877e-02
dtype: float64

In [47]:
#We can check the relation between Abdomen with other Body parts.

SlopeCheck_dataframe['ArmbyAbdomen'] = (Dataset_Train['Biceps']+Dataset_Train['Forearm']+Dataset_Train['Wrist'])/Dataset_Train['Abdomen']
SlopeCheck_dataframe['LegbyAbdomen'] = (Dataset_Train['Thigh']+Dataset_Train['Knee']+Dataset_Train['Ankle'])/Dataset_Train['Abdomen']
SlopeCheck_dataframe['NeckbyAbdomen'] = Dataset_Train['Neck']/Dataset_Train['Abdomen']

In [49]:
Y = SlopeCheck_dataframe['BodyFat']
X = SlopeCheck_dataframe.drop('BodyFat',axis=1,inplace=False)
X = sm.add_constant(X)
model = sm.OLS(Y,X,hasconst=True)
results = model.fit()

In [50]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                BodyFat   R-squared:                       0.978
Model:                            OLS   Adj. R-squared:                  0.977
Method:                 Least Squares   F-statistic:                     615.6
Date:                Wed, 07 Jul 2021   Prob (F-statistic):          1.48e-183
Time:                        20:12:24   Log-Likelihood:                 124.08
No. Observations:                 252   AIC:                            -212.2
Df Residuals:                     234   BIC:                            -148.6
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -0.1794      1.024     -0.175      0.861      -2.197       1.838
Density          -0.9361      0.019    -48.338      0.000      -0.974      -0.898
Age               0.0191      0.015      1.305      0.193      -0.010       0.048
Weight            0.0378      0.057      0.660      0.510      -0.075       0.151
Height           -0.0032      0.013     -0.249      0.804      -0.029       0.022
Neck             -0.0771      0.157     -0.491      0.624      -0.386       0.232
Chest             0.0269      0.030      0.889      0.375      -0.033       0.087
Abdomen           0.0423      0.126      0.337      0.736      -0.205       0.290
Hip               0.0165      0.038      0.431      0.667      -0.059       0.092
Thigh            -0.0137      0.105     -0.131      0.896      -0.220       0.192
Knee             -0.0033      0.049     -0.066      0.947      -0.100       0.093
Ankle            -0.0183      0.036     -0.510      0.610      -0.089       0.052
Biceps            0.0188      0.088      0.212      0.832      -0.156       0.193
Forearm           0.0308      0.053      0.582      0.561      -0.073       0.135
Wrist             0.0126      0.031      0.404      0.687      -0.049       0.074
ArmbyAbdomen     -1.1228      2.512     -0.447      0.655      -6.072       3.826
LegbyAbdomen      0.0465      1.788      0.026      0.979      -3.476       3.569
NeckbyAbdomen     2.6257      5.919      0.444      0.658      -9.035      14.286
==============================================================================
Omnibus:                      364.693   Durbin-Watson:                   2.117
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           100331.562
Skew:                           6.391   Prob(JB):                         0.00
Kurtosis:                      99.912   Cond. No.                     1.85e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.85e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [51]:
t_value = pd.DataFrame(results.tvalues,columns=['T_value'])

In [52]:
t_value.sort_values(ascending=False,by=['T_value'])

,T_value
Age,1.304583
Chest,0.889206
Weight,0.659868
Forearm,0.581813
NeckbyAbdomen,0.443626
Hip,0.430631
Wrist,0.403649
Abdomen,0.337365
Biceps,0.212474
LegbyAbdomen,0.025995
